# Serine codon landscape

Serine is the only aminoacid that is encoded by two sets of mutationally disconnected codons under the standard genetic code.

- AGY: {AGU,AGC}
- UCN: {UCA,UCC,UCG,UCU}

This leads to a landscape with two isolated fitness peaks that we can easily visualize

## 1. Defining the discrete space

The first thing we need to do is to define the discrete space for the evolutionary random walk. While we provide a generic class DiscreteSpace to define an arbitrary discrete space based on the adjacency matrix and nodes properties on which the transition between states may depend, we are going to use the class SequenceSpace that has specific built-in properties and methods specifically for sequence space.

In [1]:
# Import required libraries
import pandas as pd
from gpmap.src.space import SequenceSpace

After importing the required libraries, we can read the fitness values that we have previously generated. In this artificial example, we assigned fitnesses in the following way:

- w=2 to codons encoding Serine
- w=1 to codons encoding other aminoacides
- w=0 to stop codons

We also added some small perturbation to the fitnesses of the individual codons that could account for codon usage biases, but also allow better separation of the genotypes in the low dimensional representation. We provide different ways to generate this particular landscape

### Directly from function data


In [3]:
fpath = '../gpmap/test/data/serine.csv'
data = pd.read_csv(fpath, index_col=0)
data.head()

function
AAA  1.176405
AAC  1.040016
AAG  1.097874
AAU  1.224089
ACA  1.186756

We can see the simple table that just stores the function value for each sequence, that we will use to create a SequenceSpace object, that has a number of attributes such as number of states, allelese or the alphabet_type, in this case 'rna'

In [26]:
space = SequenceSpace(seq_length=3, alphabet_type='rna', function=data['function'])
space.n_states, space.n_alleles, space.alphabet_type

(64, [4, 4, 4], 'rna')

### Using codon model

Sometimes we may not be able to differentiate between the function or fitnesses of different codons encoding the same aminoacid, but still want to take into account the connectivity at the nucleotide level for visualizing the landscape as in a codon model of evolution.

The following table contains the fitnesses associated to each of the 20 aminoacids

In [27]:
protein_data = pd.read_csv('../gpmap/test/data/serine.protein.csv', index_col=0)
protein_data

function
protein          
A               1
C               1
D               1
E               1
F               1
G               1
H               1
I               1
K               1
L               1
M               1
N               1
P               1
Q               1
R               1
S               2
T               1
V               1
W               1
Y               1

In [28]:
space = SequenceSpace(seq_length=3, alphabet_type='rna', function=protein_data['function'],
                      codon_table='Standard', stop_function=0)
space.n_states, space.n_alleles, space.alphabet_type

(64, [4, 4, 4], 'rna')

### Using CodonSpace class

We also provide a more generic CodonSpace class that does this operation for us so that we only need to provide the aminoacid(s) are are going to be under selection, enabling also to visualizing the structure of the landscape corresponding to aminoacids with certain properties

In [29]:
from gpmap.src.space import CodonSpace

In [31]:
space = CodonSpace(allowed_aminoacids=['S'], codon_table='Standard', add_variation=True, seed=0)
space.n_states, space.n_alleles, space.alphabet_type

(64, [4, 4, 4], 'rna')

Note that we could also test how these landscapes would change under different genetic codes other than the standard. We use biopython module to translate the nucleotide sequence into protein sequence using [NCBI reference](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi) for different codon tables or genetic codes

## 2. Defining the random walk in the discrete sequence space

